# RAG System with Feedback Loop: Enhancing Retrieval and Response Quality

<img src="https://drive.google.com/uc?export=view&id=1wYSMgJtARFdvTt5g7E20mE4NmwUFUuog" width="200">

[![Build Fast with AI](https://img.shields.io/badge/BuildFastWithAI-GenAI%20Bootcamp-blue?style=for-the-badge&logo=artificial-intelligence)](https://www.buildfastwithai.com/genai-course)
[![EduChain GitHub](https://img.shields.io/github/stars/satvik314/educhain?style=for-the-badge&logo=github&color=gold)](https://github.com/satvik314/educhain)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1uNn0ojBJf4UxEfK8Q6ASOtQ8UXVKxSDv?usp=chrome_ntp)
## Master Generative AI in 6 Weeks
**What You'll Learn:**
- Build with Latest LLMs
- Create Custom AI Apps
- Learn from Industry Experts
- Join Innovation Community
Transform your AI ideas into reality through hands-on projects and expert mentorship.
[Start Your Journey](https://www.buildfastwithai.com/genai-course)
*Empowering the Next Generation of AI Innovators

Method Overview
System Initialization

The system extracts content from PDFs to build a vector database for information storage.
A retriever is configured to fetch relevant content from this vector database.
A language model (LLM) is employed to generate responses based on the retrieved data.
Query Handling

When a user submits a query, the retriever identifies and fetches the most relevant documents.
The LLM processes these documents to craft a response tailored to the query.
Collecting Feedback

Users provide feedback on the accuracy and quality of responses.
This feedback is stored in a structured format, such as a JSON file, for future reference.
Adjusting Relevance Scores

For future queries, the system evaluates stored feedback to refine the document relevance scores.
An LLM evaluates the feedback to determine its applicability to the current query.
Documents are re-ranked based on the adjusted scores.
Retriever Optimization

The retriever is updated to prioritize documents based on the adjusted scores.
This ensures that the system continuously improves its relevance for future queries.
Index Fine-Tuning

Periodically, the system fine-tunes the vector database.
High-quality feedback is used to generate additional, refined documents.
These new documents are incorporated into the vector database to further enhance retrieval accuracy.
Key Advantages
Continuous Learning: The system evolves by learning from each user interaction.
Personalization: Feedback allows the system to adapt to specific user preferences.
Improved Relevance: Responses become increasingly accurate over time due to iterative feedback integration.
Quality Assurance: Repeated low-quality responses are minimized through relevance adjustments.
Dynamic Adaptability: The system adjusts to changes in user needs and document contents.
Summary
This feedback-driven retrieval system enhances the traditional RAG (retrieval-augmented generation) pipeline by introducing a feedback loop that continuously refines the relevance of retrieved content. It is especially beneficial in domains requiring high accuracy and adaptability to evolving user requirements.




### Import relevant libraries

In [37]:
import os
import sys
from dotenv import load_dotenv
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chat_models import ChatOpenAI
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA
import json
from typing import List, Dict, Any
from langchain.embeddings.openai import OpenAIEmbeddings
from pydantic import BaseModel, Field
from langchain.prompts import PromptTemplate

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"


### Define documents path
### Download from here https://drive.google.com/file/d/1F0XGihSvR_4L0MIDH3iz02X2Ce3UcRrD/view?usp=sharing

In [72]:
path="path-to-your-file"

In [69]:
import fitz
def read_pdf_to_string(path):
    # Open the PDF document located at the specified path
    doc = fitz.open(path)
    content = ""
    # Iterate over each page in the document
    for page_num in range(len(doc)):
        # Get the current page
        page = doc[page_num]
        # Extract the text content from the current page and append it to the content string
        content += page.get_text()
    return content


In [70]:
def encode_from_string(content, chunk_size=1000, chunk_overlap=200):
    
    if not isinstance(chunk_overlap, int) or chunk_overlap < 0:
        raise ValueError("chunk_overlap must be a non-negative integer.")

    try:
        # Split the content into chunks
        text_splitter = RecursiveCharacterTextSplitter(
            chunk_size=chunk_size,
            chunk_overlap=chunk_overlap,
            length_function=len,
            is_separator_regex=False,
        )
        chunks = text_splitter.create_documents([content])

        # Assign metadata to each chunk
        for chunk in chunks:
            chunk.metadata['relevance_score'] = 1.0

        # Generate embeddings and create the vector store
        embeddings = OpenAIEmbeddings(api_key=OPENAI_API_KEY)
        vectorstore = FAISS.from_documents(chunks, embeddings)

    except Exception as e:
        raise RuntimeError(f"An error occurred during the encoding process: {str(e)}")

    return vectorstore


 Create vector store and retrieval QA chain

In [28]:
content = read_pdf_to_string(path)
vectorstore = encode_from_string(content)
retriever = vectorstore.as_retriever()

llm = ChatOpenAI(temperature=0, model_name="gpt-4o", max_tokens=4000,api_key=OPENAI_API_KEY)
qa_chain = RetrievalQA.from_chain_type(llm, retriever=retriever)

In [29]:
def get_user_feedback(query, response, relevance, quality, comments=""):
    return {
        "query": query,
        "response": response,
        "relevance": int(relevance),
        "quality": int(quality),
        "comments": comments
    }

In [68]:
def store_feedback(feedback):
    with open("feedback_data.json", "a") as f:
        json.dump(feedback, f)
        f.write("\n")

In [71]:
def load_feedback_data():
    feedback_data = []
    try:
        with open("feedback_data.json", "r") as f:
            for line in f:
                feedback_data.append(json.loads(line.strip()))
    except FileNotFoundError:
        print("No feedback data file found. Starting with empty feedback.")
    return feedback_data

### Function to adjust files relevancy based on the feedbacks file

In [57]:
class Response(BaseModel):
    answer: str = Field(..., title="The answer to the question. The options can be only 'Yes' or 'No'")

def adjust_relevance_scores(query: str, docs: List[Any], feedback_data: List[Dict[str, Any]]) -> List[Any]:
    # Create a prompt template for relevance checking
    relevance_prompt = PromptTemplate(
        input_variables=["query", "feedback_query", "doc_content", "feedback_response"],
        template="""
        Determine if the following feedback response is relevant to the current query and document content.
        You are also provided with the Feedback original query that was used to generate the feedback response.
        Current query: {query}
        Feedback query: {feedback_query}
        Document content: {doc_content}
        Feedback response: {feedback_response}
        
        Is this feedback relevant? Respond with only 'Yes' or 'No'.
        """
    )
    llm = ChatOpenAI(temperature=0, model_name="gpt-4o", max_tokens=4000,api_key=OPENAI_API_KEY)

    # Create an LLMChain for relevance checking
    relevance_chain = relevance_prompt | llm

    for doc in docs:
        relevant_feedback = []
        
        for feedback in feedback_data:
            # Use LLM to check relevance
            input_data = {
                "query": query,
                "feedback_query": feedback['query'],
                "doc_content": doc.page_content[:1000],
                "feedback_response": feedback['response']
            }
            
            result = relevance_chain.invoke(input_data).content
        
            if result.lower() == 'yes':
            
                relevant_feedback.append(feedback)
        
        # Adjust the relevance score based on feedback
        if relevant_feedback:
            avg_relevance = sum(f['relevance'] for f in relevant_feedback) / len(relevant_feedback)
            doc.metadata['relevance_score'] *= (avg_relevance / 3)  # Assuming a 1-5 scale, 3 is neutral
    
    # Re-rank documents based on adjusted scores
    return sorted(docs, key=lambda x: x.metadata['relevance_score'], reverse=True)

### Function to fine tune the vector index to include also queries + answers that received good feedbacks

In [65]:
def fine_tune_index(feedback_data: List[Dict[str, Any]], texts: List[str]) -> Any:
    # Filter high-quality responses
    good_responses = [f for f in feedback_data if f['relevance'] >= 4 and f['quality'] >= 4]
    
    # Extract queries and responses, and create new documents
    additional_texts = []
    for f in good_responses:
        combined_text = f['query'] + " " + f['response']
        additional_texts.append(combined_text)

    # make the list a string
    additional_texts = " ".join(additional_texts)
    
    # Create a new index with original and high-quality texts
    all_texts = texts + additional_texts
    new_vectorstore = encode_from_string(all_texts)
    
    return new_vectorstore

### Demonstration of how to retrieve answers with respect to user feedbacks

In [64]:

query = "What is the greenhouse effect?"

# Get response from RAG system
response = qa_chain(query)["result"]
print(response)
relevance = int(input("Enter relevance in output(out of 10): "))
quality = int(input("Enter quality in output (out of 10): "))

# Collect feedback
feedback = get_user_feedback(query, response, relevance, quality)

# Store feedback
store_feedback(feedback)

# Adjust relevance scores for future retrievals
docs = retriever.get_relevant_documents(query)
print(docs)
adjusted_docs = adjust_relevance_scores(query, docs, load_feedback_data())

# Update the retriever with adjusted docs
retriever.search_kwargs['k'] = len(adjusted_docs)
retriever.search_kwargs['docs'] = adjusted_docs

The greenhouse effect is a natural process where greenhouse gases in the Earth's atmosphere, such as carbon dioxide (CO2), methane (CH4), and nitrous oxide (N2O), trap heat from the sun. This trapped heat helps to keep the planet warm enough to support life. However, human activities have intensified this natural process by increasing the concentration of these gases in the atmosphere, leading to a warmer climate.


Enter relevance in output(out of 10):  3
Enter quality in output (out of 10):  5


[Document(id='642c992d-605c-4e4f-bd13-4ffd7a17a139', metadata={'relevance_score': 2.777777777777778}, page_content='The use of synthetic fertilizers in agriculture releases nitrous oxide, a potent greenhouse gas. \nPractices such as precision farming and organic fertilizers can mitigate these emissions. The \ndevelopment of eco-friendly fertilizers and farming techniques is essential for reducing the \nagricultural sector\'s carbon footprint. \nChapter 3: Effects of Climate Change \nThe effects of climate change are already being felt around the world and are projected to \nintensify in the coming decades. These effects include: \nRising Temperatures \nGlobal temperatures have risen by about 1.2 degrees Celsius (2.2 degrees Fahrenheit) since \nthe late 19th century. This warming is not uniform, with some regions experiencing more \nsignificant increases than others. \nHeatwaves \nHeatwaves are becoming more frequent and severe, posing risks to human health, agriculture, \nand infrastru

### Finetune the vectorstore periodicly

In [66]:
# Periodically (e.g., daily or weekly), fine-tune the index
new_vectorstore = fine_tune_index(load_feedback_data(), content)
retriever = new_vectorstore.as_retriever()